## Langchain의 Document 구조 살펴보기

In [5]:
from langchain_core.documents import Document

document = Document("안녕하세요? 이건 랭체인의 도큐먼드 입니다")
# 도큐먼트의 속성 확인
print(f'Document Architecture {document.__dict__} / type : {type(document)}')
document.metadata["source"] = "TeddyNote"
document.metadata["page"] = 1
document.metadata["author"] = "Teddy"
print(document.metadata)

Document Architecture {'id': None, 'metadata': {}, 'page_content': '안녕하세요? 이건 랭체인의 도큐먼드 입니다', 'type': 'Document'} / type : <class 'langchain_core.documents.base.Document'>
{'source': 'TeddyNote', 'page': 1, 'author': 'Teddy'}


# Document Loader
다양한 파일의 형식으로부터 불러온 내용을 문서(Document) 객체로 변환하는 역할을 합니다.

## 주요 Loader
- PyPDFLoader: PDF 파일을 로드하는 로더입니다.
- CSVLoader: CSV 파일을 로드하는 로더입니다.
- UnstructuredHTMLLoader: HTML 파일을 로드하는 로더입니다.
- JSONLoader: JSON 파일을 로드하는 로더입니다.
- TextLoader: 텍스트 파일을 로드하는 로더입니다.
- DirectoryLoader: 디렉토리를 로드하는 로더입니다.

# JSON 분할

In [ ]:
import json
from langchain_community.document_loaders import JSONLoader

def print_korean_json(data):
    if isinstance(data, dict):
        print(json.dumps(data, ensure_ascii=False, indent=2))
    elif isinstance(data, str):
        try:
            # 문자열이 실제 json이라면 파싱
            obj = json.loads(data)
            print(json.dumps(obj, ensure_ascii=False, indent=2))
        except Exception:
            # 그래도 안 되면 unicode_escape 시도
            try:
                print(data.encode('utf-8').decode('unicode_escape'))
            except Exception:
                print(data)
    else:
        print(data)

# JSONLoader 생성
loader = JSONLoader(
    file_path=r"C:\Users\cn120\바탕 화면\rag\fairness\data\dumy\total_articls.json",
    jq_schema=".[]",
    text_content=False,
)

# 문서 로드
docs = loader.load()

# 결과 출력
for doc in docs:
    doc.metadata['url'] = json.loads(doc.page_content)['URL']
    print_korean_json(doc.page_content)
    print_korean_json(doc.metadata)
    print("--------------------------------------------------------")


{
  "기사제목": "유류세 인하 8월까지 2개월 연장, 車 개소세 인하도 6개월 더 연장",
  "기사본문": "기획재정부는 16일 “6월 30일 종료될 예정이었던 유류세 한시 인하 조치를 물가 안정과 민생 지원을 위해 8월 31일까지 2개월 더 연장한다”고 밝혔다. 차량용 휘발유와 경유에는 각각 리터(L)당 820원, 581원의 유류세가 붙는데, 정부는 코로나 팬데믹 대응을 위해 유류세를 낮춘 2021년 11월부터 15차례에 걸쳐 유류세 인하 조치를 연장해 왔다.\n지난 5월 1일부터 이달 30일까지 정부는 휘발유의 경우 10%(82원) 인하된 738원, 경유는 15%(87원) 깎인 494원의 유류세를 물렸다. 정부는 앞서 인하 폭을 조정하기도 했지만 이날 16번째로 유류세 인하 조치를 연장하면서 종전 인하 폭을 유지했다. 기획재정부는 유류세 인하 조치를 연장한 이유로 최근 가공식품과 외식비를 중심으로 물가가 오르면서 민생 부담이 커지고 있다는 점과 함께 중동 사태에 따른 국제 유가 상승 가능성을 꼽았다.\n승용차 개별소비세 30% 인하 조치도 6개월 연장한다고 기재부는 밝혔다. 신차 출고가의 5%인 개별소비세 기본 세율을 3.5%로 낮추는 내용으로 감면 한도는 100만원이다. 4000만원 상당의 신차를 구매할 경우 개소세가 인하되기 전과 비교해 70만원의 세제 혜택을 본다. 정부는 2018년 7월 1일부터 승용차 개별소비세를 인하했다가 세수 부족 여파로 2023년 7월 1일부터 인하 조치를 중단했다. 하지만 작년 말부터 내수 부진이 본격화하면서 올해 1월부터 6개월간 개별소비세를 30% 낮춰줬는데, 이번에 6개월 더 연장하기로 한 것이다.",
  "신문사": "조선일보",
  "URL": "https://www.chosun.com/economy/economy_general/2025/06/16/LAXXZJ5DSFDNJJZLTL3KFNGKYA/"
}
{
  "source": "C:\\Users\\cn120\\바탕 화면\\rag\\fairness\\data\

In [ ]:
from langchain_text_splitters import RecursiveJsonSplitter

# RecursiveJsonSplitter의 split_json매소드를 이용해 json 데이터를 key-value 쌍으로 분할
# JSON 데이터를 최대 300 크기의 청크로 분할하는 RecursiveJsonSplitter 객체를 생성합니다.
splitter = RecursiveJsonSplitter(max_chunk_size=300)
with open(r"C:\Users\cn120\바탕 화면\rag\fairness\data\dumy\total_articls.json", 'r', encoding='UTF-8') as f:
    json_data = json.load(f)

# json 객체 하나씩 매핑할 것 
json_chunks = splitter.split_json(json_data=json_data[0])
for chunk in json_chunks:
    print(chunk)

{'기사제목': '유류세 인하 8월까지 2개월 연장, 車 개소세 인하도 6개월 더 연장'}
{'기사본문': '기획재정부는 16일 “6월 30일 종료될 예정이었던 유류세 한시 인하 조치를 물가 안정과 민생 지원을 위해 8월 31일까지 2개월 더 연장한다”고 밝혔다. 차량용 휘발유와 경유에는 각각 리터(L)당 820원, 581원의 유류세가 붙는데, 정부는 코로나 팬데믹 대응을 위해 유류세를 낮춘 2021년 11월부터 15차례에 걸쳐 유류세 인하 조치를 연장해 왔다.\n지난 5월 1일부터 이달 30일까지 정부는 휘발유의 경우 10%(82원) 인하된 738원, 경유는 15%(87원) 깎인 494원의 유류세를 물렸다. 정부는 앞서 인하 폭을 조정하기도 했지만 이날 16번째로 유류세 인하 조치를 연장하면서 종전 인하 폭을 유지했다. 기획재정부는 유류세 인하 조치를 연장한 이유로 최근 가공식품과 외식비를 중심으로 물가가 오르면서 민생 부담이 커지고 있다는 점과 함께 중동 사태에 따른 국제 유가 상승 가능성을 꼽았다.\n승용차 개별소비세 30% 인하 조치도 6개월 연장한다고 기재부는 밝혔다. 신차 출고가의 5%인 개별소비세 기본 세율을 3.5%로 낮추는 내용으로 감면 한도는 100만원이다. 4000만원 상당의 신차를 구매할 경우 개소세가 인하되기 전과 비교해 70만원의 세제 혜택을 본다. 정부는 2018년 7월 1일부터 승용차 개별소비세를 인하했다가 세수 부족 여파로 2023년 7월 1일부터 인하 조치를 중단했다. 하지만 작년 말부터 내수 부진이 본격화하면서 올해 1월부터 6개월간 개별소비세를 30% 낮춰줬는데, 이번에 6개월 더 연장하기로 한 것이다.'}
{'신문사': '조선일보', 'URL': 'https://www.chosun.com/economy/economy_general/2025/06/16/LAXXZJ5DSFDNJJZLTL3KFNGKYA/'}


In [42]:
# JSON 데이터를 기반으로 문서를 생성합니다.
docs = splitter.create_documents(texts=json_data)
for doc in docs:
    print(doc)

page_content='{"\uae30\uc0ac\uc81c\ubaa9": "\uc720\ub958\uc138 \uc778\ud558 8\uc6d4\uae4c\uc9c0 2\uac1c\uc6d4 \uc5f0\uc7a5, \u8eca \uac1c\uc18c\uc138 \uc778\ud558\ub3c4 6\uac1c\uc6d4 \ub354 \uc5f0\uc7a5"}'
page_content='{"\uae30\uc0ac\ubcf8\ubb38": "\uae30\ud68d\uc7ac\uc815\ubd80\ub294 16\uc77c \u201c6\uc6d4 30\uc77c \uc885\ub8cc\ub420 \uc608\uc815\uc774\uc5c8\ub358 \uc720\ub958\uc138 \ud55c\uc2dc \uc778\ud558 \uc870\uce58\ub97c \ubb3c\uac00 \uc548\uc815\uacfc \ubbfc\uc0dd \uc9c0\uc6d0\uc744 \uc704\ud574 8\uc6d4 31\uc77c\uae4c\uc9c0 2\uac1c\uc6d4 \ub354 \uc5f0\uc7a5\ud55c\ub2e4\u201d\uace0 \ubc1d\ud614\ub2e4. \ucc28\ub7c9\uc6a9 \ud718\ubc1c\uc720\uc640 \uacbd\uc720\uc5d0\ub294 \uac01\uac01 \ub9ac\ud130(L)\ub2f9 820\uc6d0, 581\uc6d0\uc758 \uc720\ub958\uc138\uac00 \ubd99\ub294\ub370, \uc815\ubd80\ub294 \ucf54\ub85c\ub098 \ud32c\ub370\ubbf9 \ub300\uc751\uc744 \uc704\ud574 \uc720\ub958\uc138\ub97c \ub0ae\ucd98 2021\ub144 11\uc6d4\ubd80\ud130 15\ucc28\ub840\uc5d0 \uac78\uccd0 \uc720\ub958\u

In [ ]:
# JSON 데이터를 기반으로 문자열 청크를 생성합니다.
# split_json이랑 차이는? 비슷하게 동작하는 걸로 보임
texts = splitter.split_text(json_data=json_data[0])
for text in texts:
    print_korean_json(text)

{
  "기사제목": "유류세 인하 8월까지 2개월 연장, 車 개소세 인하도 6개월 더 연장"
}
{
  "기사본문": "기획재정부는 16일 “6월 30일 종료될 예정이었던 유류세 한시 인하 조치를 물가 안정과 민생 지원을 위해 8월 31일까지 2개월 더 연장한다”고 밝혔다. 차량용 휘발유와 경유에는 각각 리터(L)당 820원, 581원의 유류세가 붙는데, 정부는 코로나 팬데믹 대응을 위해 유류세를 낮춘 2021년 11월부터 15차례에 걸쳐 유류세 인하 조치를 연장해 왔다.\n지난 5월 1일부터 이달 30일까지 정부는 휘발유의 경우 10%(82원) 인하된 738원, 경유는 15%(87원) 깎인 494원의 유류세를 물렸다. 정부는 앞서 인하 폭을 조정하기도 했지만 이날 16번째로 유류세 인하 조치를 연장하면서 종전 인하 폭을 유지했다. 기획재정부는 유류세 인하 조치를 연장한 이유로 최근 가공식품과 외식비를 중심으로 물가가 오르면서 민생 부담이 커지고 있다는 점과 함께 중동 사태에 따른 국제 유가 상승 가능성을 꼽았다.\n승용차 개별소비세 30% 인하 조치도 6개월 연장한다고 기재부는 밝혔다. 신차 출고가의 5%인 개별소비세 기본 세율을 3.5%로 낮추는 내용으로 감면 한도는 100만원이다. 4000만원 상당의 신차를 구매할 경우 개소세가 인하되기 전과 비교해 70만원의 세제 혜택을 본다. 정부는 2018년 7월 1일부터 승용차 개별소비세를 인하했다가 세수 부족 여파로 2023년 7월 1일부터 인하 조치를 중단했다. 하지만 작년 말부터 내수 부진이 본격화하면서 올해 1월부터 6개월간 개별소비세를 30% 낮춰줬는데, 이번에 6개월 더 연장하기로 한 것이다."
}
{
  "신문사": "조선일보",
  "URL": "https://www.chosun.com/economy/economy_general/2025/06/16/LAXXZJ5DSFDNJJZLTL3KFNGKYA/"
}


## 텍스트 분할

In [77]:
from langchain_text_splitters import CharacterTextSplitter

with open(r"C:\Users\cn120\바탕 화면\rag\fairness\data\dumy\total_articls.json", 'r', encoding='UTF-8') as f:
    json_data = json.load(f)

text_splitter = CharacterTextSplitter(
    # 텍스트를 분할할 때 사용할 구분자를 지정합니다. 기본값은 "\n\n"입니다.
    # separator= " ",
    # 분할된 텍스트 청크의 최대 크기(글자 갯수)를 지정합니다.
    chunk_size=3000,
    # 분할된 텍스트 청크 간의 중복되는 문자 수를 지정합니다.
    chunk_overlap=50,
    # 텍스트의 길이를 계산하는 함수를 지정합니다.
    length_function=len,
    # 구분자가 정규식인지 여부를 지정합니다.
    is_separator_regex=False,
)

# text_splitter를 사용하여 state_of_the_union 텍스트를 문서로 분할합니다.
texts = text_splitter.create_documents([json_data[i]['기사본문'] for i in range(len(json_data))])
for text in texts:
    print(text)

page_content='기획재정부는 16일 “6월 30일 종료될 예정이었던 유류세 한시 인하 조치를 물가 안정과 민생 지원을 위해 8월 31일까지 2개월 더 연장한다”고 밝혔다. 차량용 휘발유와 경유에는 각각 리터(L)당 820원, 581원의 유류세가 붙는데, 정부는 코로나 팬데믹 대응을 위해 유류세를 낮춘 2021년 11월부터 15차례에 걸쳐 유류세 인하 조치를 연장해 왔다.
지난 5월 1일부터 이달 30일까지 정부는 휘발유의 경우 10%(82원) 인하된 738원, 경유는 15%(87원) 깎인 494원의 유류세를 물렸다. 정부는 앞서 인하 폭을 조정하기도 했지만 이날 16번째로 유류세 인하 조치를 연장하면서 종전 인하 폭을 유지했다. 기획재정부는 유류세 인하 조치를 연장한 이유로 최근 가공식품과 외식비를 중심으로 물가가 오르면서 민생 부담이 커지고 있다는 점과 함께 중동 사태에 따른 국제 유가 상승 가능성을 꼽았다.
승용차 개별소비세 30% 인하 조치도 6개월 연장한다고 기재부는 밝혔다. 신차 출고가의 5%인 개별소비세 기본 세율을 3.5%로 낮추는 내용으로 감면 한도는 100만원이다. 4000만원 상당의 신차를 구매할 경우 개소세가 인하되기 전과 비교해 70만원의 세제 혜택을 본다. 정부는 2018년 7월 1일부터 승용차 개별소비세를 인하했다가 세수 부족 여파로 2023년 7월 1일부터 인하 조치를 중단했다. 하지만 작년 말부터 내수 부진이 본격화하면서 올해 1월부터 6개월간 개별소비세를 30% 낮춰줬는데, 이번에 6개월 더 연장하기로 한 것이다.'
page_content='정부가 내수와 민생 경기 회복을 위해 1인당 최대 40만원대의 민생 회복 지원금을 지급하는 내용의 추가경정예산안을 조만간 마련해 발표할 것으로 알려졌다. 산불 등 재해·재난 대응과 통상 경쟁력 강화 등을 위해 지난달 1일 편성된 13조8000억원의 ‘필수 추경’에 이어 올해 두 번째 추경이다. 전국 지자체의 지역화폐(지역사랑상품권) 발행을 위한 국비 지원액을 5000억원 이상 추가로 